IMPORTAMOS PANDAS PARA ABRIR EL DATASET.

In [1]:
import pandas as pd

In [2]:
df_games=pd.read_json(r'C:\Users\User\OneDrive\Documents\Trabajo\Henry\Proyecto Ind. 1\Datos Crudos\output_steam_games.json', lines=True)

SE PUEDE OBSERVAR UNA GRAN CANTIDAD DE DATOS NULOS, ACONTINUACION LIMPIARE LOS DATOS NULOS DEL DATASET

In [3]:
df_games_full=df_games.dropna(subset='id', axis=0)
df_games_full.shape

(32133, 13)

In [4]:
columnas_no=['publisher','app_name','url','tags','specs','early_access','reviews_url']
columnas_si=['genres','title','release_date','price','id','developer']

In [5]:
df_games_full.drop(columns=columnas_no, inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_19648\3064507500.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_full.drop(columns=columnas_no, inplace=True)


In [6]:
df_games_full.dropna(subset=columnas_si, inplace=True)
df_games_full.shape

C:\Users\User\AppData\Local\Temp\ipykernel_19648\80426719.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_full.dropna(subset=columnas_si, inplace=True)


(27462, 6)

CONVERTIMOS LOS DATOS "FREE TO PLAY" DE LA COLUMNA "PRICE" A NUMEROS FLOTANTES

In [7]:
def precio_float(precio: str) -> float:
    try:
        return float(precio)
    except (ValueError, TypeError):
        return 0.0

In [8]:
df_games_full['price']=df_games_full['price'].apply(precio_float)

C:\Users\User\AppData\Local\Temp\ipykernel_19648\519369855.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_full['price']=df_games_full['price'].apply(precio_float)


AHORA EXTRAEMOS DE LA COLUMNA "RELEASE_DATE" SOLAMENTE EL ANIO, A CONTINUACION EL PROCESO DE EXTRACCION

In [9]:
df_games_full['release_date']=df_games_full['release_date'].astype(str)

C:\Users\User\AppData\Local\Temp\ipykernel_19648\1971319149.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_full['release_date']=df_games_full['release_date'].astype(str)


In [10]:
import re #pip install regex

In [11]:
def solo_anio(cadena: str) -> str|None: 
    find=re.search(r'(\d{4})', str(cadena))

    if find:
        return find.group(1)
    else:
        return None

In [12]:
df_games_full['release_date']=df_games_full['release_date'].apply(solo_anio)

C:\Users\User\AppData\Local\Temp\ipykernel_19648\2999718140.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_full['release_date']=df_games_full['release_date'].apply(solo_anio)


PARA MAYOR PROLIJIDAD, CAMBIAMOS EL NOMBRE DE LA COLUMNA "ID" POR "ITEM_ID"

In [13]:
df_games_full.rename(columns={'id':'item_id'},inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_19648\3306949731.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_full.rename(columns={'id':'item_id'},inplace=True)


EXTRAEMOS TODOS LOS ITEMS DENTRO DE LA LISTA DE CADA FILA DE LA COLUMNA "GENRES" A UN ITEM ACOMODADO POR FILA

In [14]:
df_games_full=df_games_full.explode('genres')
df_games_full.head()

,genres,title,release_date,price,item_id,developer
88310,Action,Lost Summoner Kitty,2018,4.99,761140.0,Kotoshiro
88310,Casual,Lost Summoner Kitty,2018,4.99,761140.0,Kotoshiro
88310,Indie,Lost Summoner Kitty,2018,4.99,761140.0,Kotoshiro
88310,Simulation,Lost Summoner Kitty,2018,4.99,761140.0,Kotoshiro
88310,Strategy,Lost Summoner Kitty,2018,4.99,761140.0,Kotoshiro


EXPORTAMOS TODOS LOS DATOS NORMALLIZADOS A UN ARCHIVO .PARQUET

In [15]:
df_games_full.to_parquet(r'C:\Users\User\OneDrive\Documents\Trabajo\Henry\Proyecto Ind. 1\Datos Limpios\steam_games.parquet', compression='gzip')